## Benchmark Nixtla implementation single-process on M5

In [1]:
!pip install hierarchicalforecast
!pip install -U numba statsforecast datasetsforecast

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [51]:
import numpy as np
import pandas as pd

#obtain hierarchical data
from datasetsforecast.hierarchical import HierarchicalData

# compute base forecast no coherent
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive

#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut

from timeit import default_timer as timer

ROOT = "/data/cmu/large-scale-hts-reconciliation/"
data_dir = ROOT + "notebooks/"

In [52]:
import pickle

tags = pickle.load(open(data_dir + 'm5_prediction_raw/tags.pkl', 'rb'))
S_df = pickle.load(open(data_dir + 'm5_prediction_raw/nixtla_s.pkl', 'rb'))
Y_df = pd.read_csv(data_dir + 'm5_prediction_raw/nixtla_y_gt.csv')
Y_hat_df = pd.read_csv(data_dir + 'm5_prediction_raw/nixtla_y_hat.csv')

#Y_df, S_df, tags = HierarchicalData.load('./data', 'TourismSmall')

start = pd.to_datetime("2016-01-01")
    
Y_df['ds'] = pd.to_datetime(Y_df['ds'].apply(lambda x: start + pd.Timedelta(days = int(x[2:]))))

In [53]:
Y_df.set_index('unique_id', inplace=True)
Y_hat_df.set_index('unique_id', inplace=True)

In [54]:
Y_df.head()

,ds,y
unique_id,,
HOBBIES_1_001_CA_1,2021-01-16,4.0
HOBBIES_1_001_CA_1,2021-01-17,0.0
HOBBIES_1_001_CA_1,2021-01-18,0.0
HOBBIES_1_001_CA_1,2021-01-19,0.0
HOBBIES_1_001_CA_1,2021-01-20,0.0


In [55]:
S_df.iloc[:5, :5]

,HOBBIES_1_001_CA_1,HOBBIES_1_002_CA_1,HOBBIES_1_003_CA_1,HOBBIES_1_004_CA_1,HOBBIES_1_005_CA_1
HOBBIES,1,1,1,1,1
HOUSEHOLD,0,0,0,0,0
FOODS,0,0,0,0,0
HOBBIES_1,1,1,1,1,1
HOBBIES_2,0,0,0,0,0


In [56]:
tags

{'Cat': array(['HOBBIES', 'HOUSEHOLD', 'FOODS'], dtype=object),
 'Cat/Dept': array(['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1',
        'FOODS_2', 'FOODS_3'], dtype=object),
 'Cat/Dept/Item': array(['HOBBIES_1_001', 'HOBBIES_1_002', 'HOBBIES_1_003', ...,
        'FOODS_3_825', 'FOODS_3_826', 'FOODS_3_827'], dtype=object),
 'Cat/Dept/Item/State': array(['HOBBIES_1_001_CA_1', 'HOBBIES_1_002_CA_1', 'HOBBIES_1_003_CA_1',
        ..., 'FOODS_3_825_WI_3', 'FOODS_3_826_WI_3', 'FOODS_3_827_WI_3'],
       dtype=object)}

In [57]:
Y_hat_df['ds'] = pd.to_datetime(Y_hat_df['ds'].apply(lambda x: start + pd.Timedelta(days = int(x[2:]))))
Y_hat_df.head()

,ds,prophet
unique_id,,
HOBBIES_1_001_CA_1,2021-01-16,0.641144
HOBBIES_1_001_CA_1,2021-01-17,0.855605
HOBBIES_1_001_CA_1,2021-01-18,0.713611
HOBBIES_1_001_CA_1,2021-01-19,1.213304
HOBBIES_1_001_CA_1,2021-01-20,1.142696


The following cell makes the previous forecasts coherent using the `HierarchicalReconciliation` class. The used methods to make the forecasts coherent are:
- `BottomUp`: The reconciliation of the method is a simple addition to the upper levels.
- `TopDown`: The second method constrains the base-level predictions to the top-most aggregate-level serie and then distributes it to the disaggregate series through the use of proportions. 
- `MiddleOut`: Anchors the base predictions in a middle level.

In [ ]:
date = '2021-01-16'

Y_hat = Y_hat_df.loc[Y_hat_df['ds'] == date]
Y = Y_df.loc[Y_df['ds'] == date]

reconcilers = [
    BottomUp(),
    #TopDown(method='forecast_proportions'),
    #MiddleOut(middle_level='Country/Purpose/State', 
    #          top_down_method='forecast_proportions')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)

start = timer()
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat, Y_df=Y, 
                          S=S_df, tags=tags)
end = timer()
print(end - start)

In [43]:
Y_hat

,unique_id,ds,prophet
0,HOBBIES_1_001_CA_1,2021-01-16,0.641144
100,HOBBIES_1_002_CA_1,2021-01-16,0.226241
200,HOBBIES_1_003_CA_1,2021-01-16,0.407938
300,HOBBIES_1_004_CA_1,2021-01-16,2.470077
400,HOBBIES_1_005_CA_1,2021-01-16,1.359481
...,...,...,...
3048500,FOODS_3_823_WI_3,2021-01-16,0.464440
3048600,FOODS_3_824_WI_3,2021-01-16,0.398286
3048700,FOODS_3_825_WI_3,2021-01-16,1.145024
3048800,FOODS_3_826_WI_3,2021-01-16,1.467588


In [42]:
S_df

,HOBBIES_1_001_CA_1,HOBBIES_1_002_CA_1,HOBBIES_1_003_CA_1,HOBBIES_1_004_CA_1,HOBBIES_1_005_CA_1,HOBBIES_1_006_CA_1,HOBBIES_1_007_CA_1,HOBBIES_1_008_CA_1,HOBBIES_1_009_CA_1,HOBBIES_1_010_CA_1,...,FOODS_3_818_WI_3,FOODS_3_819_WI_3,FOODS_3_820_WI_3,FOODS_3_821_WI_3,FOODS_3_822_WI_3,FOODS_3_823_WI_3,FOODS_3_824_WI_3,FOODS_3_825_WI_3,FOODS_3_826_WI_3,FOODS_3_827_WI_3
HOBBIES,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
HOUSEHOLD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FOODS,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
HOBBIES_1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
HOBBIES_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOODS_3_823_WI_3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
FOODS_3_824_WI_3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
FOODS_3_825_WI_3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
FOODS_3_826_WI_3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [34]:
a,b,c = HierarchicalData.load('./data', 'TourismSmall')

100%|████████████████████████████████████████████████████████████| 1.30M/1.30M [00:00<00:00, 4.27MiB/s]
INFO:datasetsforecast.utils:Successfully downloaded datasets.zip, 1297279, bytes.
INFO:datasetsforecast.utils:Decompressing zip file...
INFO:datasetsforecast.utils:Successfully decompressed data/hierarchical/datasets.zip


In [45]:
a['unique_id'].unique()

array(['total', 'hol', 'vfr', 'bus', 'oth', 'nsw-hol', 'vic-hol',
       'qld-hol', 'sa-hol', 'wa-hol', 'tas-hol', 'nt-hol', 'nsw-vfr',
       'vic-vfr', 'qld-vfr', 'sa-vfr', 'wa-vfr', 'tas-vfr', 'nt-vfr',
       'nsw-bus', 'vic-bus', 'qld-bus', 'sa-bus', 'wa-bus', 'tas-bus',
       'nt-bus', 'nsw-oth', 'vic-oth', 'qld-oth', 'sa-oth', 'wa-oth',
       'tas-oth', 'nt-oth', 'nsw-hol-city', 'nsw-hol-noncity',
       'vic-hol-city', 'vic-hol-noncity', 'qld-hol-city',
       'qld-hol-noncity', 'sa-hol-city', 'sa-hol-noncity', 'wa-hol-city',
       'wa-hol-noncity', 'tas-hol-city', 'tas-hol-noncity', 'nt-hol-city',
       'nt-hol-noncity', 'nsw-vfr-city', 'nsw-vfr-noncity',
       'vic-vfr-city', 'vic-vfr-noncity', 'qld-vfr-city',
       'qld-vfr-noncity', 'sa-vfr-city', 'sa-vfr-noncity', 'wa-vfr-city',
       'wa-vfr-noncity', 'tas-vfr-city', 'tas-vfr-noncity', 'nt-vfr-city',
       'nt-vfr-noncity', 'nsw-bus-city', 'nsw-bus-noncity',
       'vic-bus-city', 'vic-bus-noncity', 'qld-bus-city'

In [38]:
tags

{'level_1': array(['HOBBIES', 'HOUSEHOLD', 'FOODS'], dtype=object),
 'level_2': array(['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1',
        'FOODS_2', 'FOODS_3'], dtype=object),
 'level_3': array(['HOBBIES_1_001', 'HOBBIES_1_002', 'HOBBIES_1_003', ...,
        'FOODS_3_825', 'FOODS_3_826', 'FOODS_3_827'], dtype=object),
 'level_4': array(['HOBBIES_1_001_CA_1', 'HOBBIES_1_002_CA_1', 'HOBBIES_1_003_CA_1',
        ..., 'FOODS_3_825_WI_3', 'FOODS_3_826_WI_3', 'FOODS_3_827_WI_3'],
       dtype=object)}

In [39]:
c

{'Country': array(['total'], dtype=object),
 'Country/Purpose': array(['hol', 'vfr', 'bus', 'oth'], dtype=object),
 'Country/Purpose/State': array(['nsw-hol', 'vic-hol', 'qld-hol', 'sa-hol', 'wa-hol', 'tas-hol',
        'nt-hol', 'nsw-vfr', 'vic-vfr', 'qld-vfr', 'sa-vfr', 'wa-vfr',
        'tas-vfr', 'nt-vfr', 'nsw-bus', 'vic-bus', 'qld-bus', 'sa-bus',
        'wa-bus', 'tas-bus', 'nt-bus', 'nsw-oth', 'vic-oth', 'qld-oth',
        'sa-oth', 'wa-oth', 'tas-oth', 'nt-oth'], dtype=object),
 'Country/Purpose/State/CityNonCity': array(['nsw-hol-city', 'nsw-hol-noncity', 'vic-hol-city',
        'vic-hol-noncity', 'qld-hol-city', 'qld-hol-noncity',
        'sa-hol-city', 'sa-hol-noncity', 'wa-hol-city', 'wa-hol-noncity',
        'tas-hol-city', 'tas-hol-noncity', 'nt-hol-city', 'nt-hol-noncity',
        'nsw-vfr-city', 'nsw-vfr-noncity', 'vic-vfr-city',
        'vic-vfr-noncity', 'qld-vfr-city', 'qld-vfr-noncity',
        'sa-vfr-city', 'sa-vfr-noncity', 'wa-vfr-city', 'wa-vfr-noncity',
     